In [2]:
import pymongo
import ssl
import pandas as pd
import datetime
from bson.decimal128 import Decimal128
from bson.son import SON
import uuid
import pprint

In [3]:
# mongo_uri="mongodb://{username}@mongo-db00.dev.cld.dataeng.internal:27017/?authMechanism={auth_mechanism}&replicaSet={replica_set}&readPreference=primary&ssl={ssl}&tlsAllowInvalidCertificates=true&authSource=%24external"
#mongo_uri="mongodb://{username}@{mongo_hosts}/?authMechanism={auth_mechanism}&replicaSet={replica_set}&ssl={ssl}&tlsAllowInvalidCertificates=true&authSource=%24external"
# &readPreference=primary

mongo_uri="mongodb://{username}@{mongo_hosts}/?authMechanism=GSSAPI"
env="prod"

svx_replica_set = {
    "dev":"svx-dev1",
    "pre":"svx-preprod1",
    "prod":"svx-prod1"
}
mongo_hosts = ["mongo-db00.{env}.cld.dataeng.internal:27017".format(env=env),
               "mongo-db01.{env}.cld.dataeng.internal:27017".format(env=env),
               "mongo-db02.{env}.cld.dataeng.internal:27017".format(env=env)
              ]

svx_replica_set = svx_replica_set.get(env)
username = "s120748@AUIAG.CORP"

client=pymongo.MongoClient(
    host=mongo_hosts,
    replicaset=svx_replica_set,
    ssl=True,
    ssl_cert_reqs=ssl.CERT_NONE,
    username=username,
    authMechanism='GSSAPI',
    authSource='$external',
    readPreference="primaryPreferred",
    w="majority",
    j=True,
    retryWrites=True
    )


In [4]:
db = client.svx

In [5]:
db.list_collection_names()

['online_events',
 'customer_complaints',
 'policy_customer',
 'job_stats',
 'ds_svx_iag_complaints_data_harmonized',
 'online_events_staging',
 'rt_svx_huon_policy',
 'customer_activity',
 'svx_iag_complaints_data_harmonized_staging',
 'properties_staging',
 'online_activities_staging',
 'consumer_invitation_responses',
 'customer_insights',
 'ds_pi_claims',
 'ds_ci_claims',
 'online_activities',
 'mvdetail',
 'data_provenances',
 'customer_address',
 'stg_customer',
 'vehicles_previous',
 'system.profile',
 'properties',
 'customer_analytical_profiles',
 'products',
 'ds_crods_src_sys_bus',
 'ds_svx_cap_customer',
 'ds_sor_onthehouse',
 'ds_svx_cap_customer_staging',
 'ds_svx_cap_policy',
 'ds_experian_ext_iag_pid',
 'ds_iag_product_cover',
 'property_structures',
 'ds_mdl_policy',
 'customer',
 'vehicles',
 'policy_insights',
 'external_cache',
 'ccpi-vehicle-view']

In [6]:
collection = db.data_provenances

In [7]:
query = {
    'event': 'CREATE',
    'created_datetime': {
        '$gte': datetime.datetime(2021, 3, 21),
        '$lt': datetime.datetime(2021, 3, 22)
    }
}

cursor = collection.find(query)
result = list(cursor)
df = pd.DataFrame(result)
df

,_id,request_url,source_object_type,event,created_by_user,content,provenance_id,created_datetime
0,60568c8360e7c9c63605ebae,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GASA_414953016', 'linked_address_ids'...",a8847165-c2dc-4d4b-9191-1da8b3cff767,2021-03-21 00:00:03.810
1,60568c88c719d0c6e119f6e2,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,sys-buy-journey-nrma-user,"{'_id': 'GANSW704205437', 'linked_address_ids'...",8b0d19f4-dbc8-474c-9ee9-95ac94bde48f,2021-03-21 00:00:08.695
2,60568c8abba0b3eef5cf1dd1,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GANT_718393278', 'linked_address_ids'...",f2c1acfc-2a80-4f1a-b9ea-5c6543bbcb5f,2021-03-21 00:00:10.608
3,60568c8c60e7c9c63605ebaf,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GASA_425063612', 'linked_address_ids'...",20693f03-a3ac-49aa-93d2-b67f66cc99df,2021-03-21 00:00:12.522
4,60568c92d57d0aeb7c020c3a,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GASA_415034033', 'linked_address_ids'...",71ef1c48-a7a4-4bf8-a3fe-aa3f5db859f6,2021-03-21 00:00:18.635
...,...,...,...,...,...,...,...,...
8092,6057ddfabba0b3eef5cf28e3,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GANT_703852995', 'linked_address_ids'...",243729ea-8c19-49a8-b9a4-dbc71adfef0e,2021-03-21 23:59:54.713
8093,6057ddfbc719d0c6e11a01c7,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GASA_415074597', 'linked_address_ids'...",abc3e5b4-8bbf-4c17-b622-492dfedcd205,2021-03-21 23:59:55.471
8094,6057ddfc60e7c9c63605f5df,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,fireflysystemuser,"{'_id': 'GASA_414947893', 'linked_address_ids'...",4cfa2136-0ccf-49a3-ab11-b9dcb73b7db2,2021-03-21 23:59:56.757
8095,6057ddfd60e7c9c63605f5e0,https://svx.dataeng.internal/properties?addres...,PROPERTY,CREATE,sys-buy-journey-nrma-user,"{'_id': 'GAQLD162715298', 'linked_address_ids'...",4a956c18-25c1-4cc5-8da0-2563d6b9c20c,2021-03-21 23:59:57.679


In [13]:
df.groupby(df.created_by_user) \
  .agg({
        '_id': ['count']
    })

,_id
,count
created_by_user,
fireflysystemuser,5967
sys-buy-journey-nrma-user,2130


In [15]:
query = {
    'provenance_id': uuid.UUID('d52b40e9-fb64-4bb4-897d-83c910c2c31c')
}

In [17]:
cursor = collection.find(query,
                                 sort=[('created_datetime', -1)]
                                )

Property structures collection

In [8]:
collection = db['property_structures']

In [24]:
# number of levels
pipeline = [
    {"$group": {"_id": "$number_of_levels", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 14790232},
 {'_id': 1.0, 'count': 1453563},
 {'_id': 2.0, 'count': 481229},
 {'_id': 3.0, 'count': 44713},
 {'_id': 1.5, 'count': 14417},
 {'_id': 4.0, 'count': 4608},
 {'_id': 5.0, 'count': 431},
 {'_id': 6.0, 'count': 338},
 {'_id': 8.0, 'count': 303},
 {'_id': 7.0, 'count': 260},
 {'_id': 9.0, 'count': 169}]
FAIL: None does not exist in the allowed list of values.
FAIL: 1.5 does not exist in the allowed list of values.


In [156]:
# structure type
pipeline = [
    {"$group": {"_id": "$structure_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Apartment', 'BED AND BREAKFAST', 'Commercial Farm', 'Commercial Office', 'DUPLEX', 
                'Factory Unit', 'Flat', 'GRANNY FLAT - SELF CONTAINED', 
                'HOBBY FARM OR A DOMESTIC DWELLING ON A COMMERCIAL PROPERTY', 
                'HOUSE (FREESTANDING)/TERRACE HOUSE FREE STANDING HOUSE', 'House', 'MISSING', 
                'RELOCATABLE / DEMOUNTABLE DWELLING', 'RETIREMENT VILLAGE', 'Retirement Village', 
                'SEMI-DETACHED HOUSE OR TERRACE', 'SHOPFRONT', 'STUDENT QUARTERS / NURSES QTRS', 'Semi-detached', 
                'Shed', 'Shop', 'TOWNHOUSE', 'TOWNHOUSE, VILLA OR DUPLEX', 'Terraced', 'Townhouse', 
                'UNIT/FLAT/APARTMENT (PART OF ABLOCK) UNIT OR FLAT', 'UNITS (BLOCK OF)', 'Unit', 'VILLA', 'Warehouse']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 14934903},
 {'_id': 'Normal Dwelling', 'count': 469499},
 {'_id': 'HOUSE (FREESTANDING)/TERRACE HOUSE FREE STANDING HOUSE',
  'count': 410723},
 {'_id': 'House', 'count': 373172},
 {'_id': 'UNIT/FLAT/APARTMENT (PART OF ABLOCK) UNIT OR FLAT', 'count': 213460},
 {'_id': 'TOWNHOUSE, VILLA OR DUPLEX', 'count': 113084},
 {'_id': 'MISSING', 'count': 55839},
 {'_id': 'Home Unit', 'count': 53372},
 {'_id': 'Townhouse', 'count': 35762},
 {'_id': 'SEMI-DETACHED HOUSE OR TERRACE', 'count': 14173},
 {'_id': 'Semi Detached Dwelling', 'count': 7675},
 {'_id': 'Holiday Home - Weekender', 'count': 6001},
 {'_id': 'Apartment', 'count': 5441},
 {'_id': 'Flat / Apartment', 'count': 4785},
 {'_id': 'Retirement Village Unit', 'count': 3405},
 {'_id': 'Semi-detached', 'count': 3006},
 {'_id': 'Unit', 'count': 2786},
 {'_id': 'Terrace House', 'count': 1461},
 {'_id': 'Terraced', 'count': 1432},
 {'_id': 'TOWNHOUSE', 'count': 901},
 {'_id': 'Unit/Flat', 'count': 811},
 {'_id': 'Block o

In [157]:
# occupancy_code
pipeline = [
    {"$group": {"_id": "$occupancy_code", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'COMBINATION OF OWNER(S) AND BOARDER(S)', 'COMBINATION OF OWNER(S) AND TENANT(S)', 
                'FOR SALE AND TENANTED', 'I own and live in the home', 
                'I rent from a landlord, by myself or with family', "It's my holiday/weekend home", 
                "It's my holiday/weekend home that I also rent out", 'No one lives in the home',
                'OWNER WHO LIVES IN THE HOME', "OWNER'S HOLIDAY HOME/WEEKENDER", 
                'RENTER - INSURING OWN CONTENTS LEASED TO TENANT', 'SEARCHING FOR TENANT UNOCCUPIED <= 30 DAYS', 
                'STRATA OWNER WHO LIVES IN THEHOME', 'STRATA TENANTS WHO LIVE IN THEHOME', 
                'TENANTS - LANDLORD INSURING CONTENTS', 'TENANTS - OWNER RENTS HOME OUTFOR HOLIDAY LETTING', 
                'UNOCCUPIED']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 13183560},
 {'_id': 'OWNER WHO LIVES IN THE HOME', 'count': 1898152},
 {'_id': 'RENTER - INSURING OWN CONTENTS LEASED TO TENANT', 'count': 798230},
 {'_id': 'Owner Occupier', 'count': 273944},
 {'_id': 'TENANTS - LANDLORD INSURING CONTENTS', 'count': 193566},
 {'_id': 'Tenants', 'count': 98251},
 {'_id': 'Tenants via a landlord', 'count': 70925},
 {'_id': 'COMBINATION OF OWNER(S) AND TENANT(S)', 'count': 38333},
 {'_id': 'UNOCCUPIED', 'count': 37935},
 {'_id': 'Unknown', 'count': 32982},
 {'_id': "OWNER'S HOLIDAY HOME/WEEKENDER", 'count': 29276},
 {'_id': 'SEARCHING FOR TENANT UNOCCUPIED <= 30 DAYS', 'count': 22783},
 {'_id': 'I own and live in the home', 'count': 9372},
 {'_id': 'TENANTS - OWNER RENTS HOME OUTFOR HOLIDAY LETTING', 'count': 9194},
 {'_id': 'STRATA OWNER WHO LIVES IN THEHOME', 'count': 5761},
 {'_id': 'Owners Holiday Home / Weekender', 'count': 5089},
 {'_id': 'I rent from a landlord with flatmates', 'count': 2483},
 {'_id': 'Unoccupied', 'count'

In [125]:
# primary roof type
pipeline = [
    {"$group": {"_id": "$primary_roof_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Asbestos', 'Concrete roof', 'Concrete tile', 'Fibrecement covering', 'Metal covering', 
                'Slate', 'Terracotta', 'Timber shingles']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 14467428},
 {'_id': 'Concrete tile', 'count': 1067735},
 {'_id': 'Metal covering', 'count': 719830},
 {'_id': 'Terracotta', 'count': 423528},
 {'_id': 'Slate', 'count': 18691},
 {'_id': 'Fibrecement covering', 'count': 12480},
 {'_id': 'Timber shingles', 'count': 4808},
 {'_id': 'Asbestos', 'count': 1249},
 {'_id': 'Concrete roof', 'count': 805},
 {'_id': 'Other', 'count': 26}]
FAIL: Other does not exist in the allowed list of values.


In [126]:
# primary wall type
pipeline = [
    {"$group": {"_id": "$primary_wall_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Blockwork', 'Brick veneer', 'Double brick', 'Other', 'Rendered masonry', 'Sheet cladding', 
                'Weatherboard - Artificial', 'Weatherboard - Timber']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))
    
x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 13907511},
 {'_id': 'Brick veneer', 'count': 1577772},
 {'_id': 'Double brick', 'count': 542154},
 {'_id': 'Weatherboard - Timber', 'count': 308571},
 {'_id': 'Weatherboard - Artificial', 'count': 139707},
 {'_id': 'Sheet cladding', 'count': 129573},
 {'_id': 'Other', 'count': 63160},
 {'_id': 'Blockwork', 'count': 45932},
 {'_id': 'Rendered masonry', 'count': 2013},
 {'_id': 'Stonework veneer', 'count': 109},
 {'_id': 'Stonework', 'count': 77},
 {'_id': 'AAC panel', 'count': 1}]
FAIL: Stonework veneer does not exist in the allowed list of values.
FAIL: Stonework does not exist in the allowed list of values.
FAIL: AAC panel does not exist in the allowed list of values.


In [130]:
# period of construction 
pipeline = [
    {"$group": {"_id": "$period_of_construction", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Contemporary", "Federation", "PostWar", "Victorian", "War"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 14419731},
 {'_id': 'Contemporary', 'count': 1929123},
 {'_id': 'PostWar', 'count': 174228},
 {'_id': 'War', 'count': 153931},
 {'_id': 'Federation', 'count': 27128},
 {'_id': 'Victorian', 'count': 12439}]
SUCCESS


In [131]:
# quality of construction 
pipeline = [
    {"$group": {"_id": "$quality_of_construction", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Quality", "Standard"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15475363},
 {'_id': 'Standard', 'count': 1217132},
 {'_id': 'Quality', 'count': 24084},
 {'_id': 'Prestige', 'count': 1}]
FAIL: Prestige does not exist in the allowed list of values.


In [25]:
# bedroom count
pipeline = [
    {"$group": {"_id": "$bedroom_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15353648},
 {'_id': 3, 'count': 571148},
 {'_id': 4, 'count': 492488},
 {'_id': 5, 'count': 167045},
 {'_id': 2, 'count': 115898},
 {'_id': 6, 'count': 34502},
 {'_id': 0, 'count': 29891},
 {'_id': 1, 'count': 14144},
 {'_id': 7, 'count': 6663},
 {'_id': 8, 'count': 4820},
 {'_id': 9, 'count': 8},
 {'_id': 10, 'count': 2},
 {'_id': 54, 'count': 1},
 {'_id': 41, 'count': 1},
 {'_id': 36, 'count': 1},
 {'_id': 18, 'count': 1},
 {'_id': 12, 'count': 1},
 {'_id': 11, 'count': 1}]
FAIL: None does not exist in the allowed list of values.


In [27]:
# bedroom small count
pipeline = [
    {"$group": {"_id": "$bedroom_small_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(1, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16784162},
 {'_id': 0, 'count': 5731},
 {'_id': 1, 'count': 160},
 {'_id': 2, 'count': 122},
 {'_id': 3, 'count': 61},
 {'_id': 4, 'count': 21},
 {'_id': 20, 'count': 3},
 {'_id': 9, 'count': 1},
 {'_id': 7, 'count': 1},
 {'_id': 6, 'count': 1}]
FAIL: None does not exist in the allowed list of values.
FAIL: 0 does not exist in the allowed list of values.


In [28]:
# bedroom medium count
pipeline = [
    {"$group": {"_id": "$bedroom_medium_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15567096},
 {'_id': 3, 'count': 508295},
 {'_id': 4, 'count': 431586},
 {'_id': 5, 'count': 136053},
 {'_id': 2, 'count': 87599},
 {'_id': 6, 'count': 27223},
 {'_id': 0, 'count': 13077},
 {'_id': 1, 'count': 9817},
 {'_id': 7, 'count': 5119},
 {'_id': 8, 'count': 4079},
 {'_id': 10, 'count': 220},
 {'_id': 20, 'count': 28},
 {'_id': 9, 'count': 16},
 {'_id': 13, 'count': 10},
 {'_id': 17, 'count': 7},
 {'_id': 15, 'count': 7},
 {'_id': 14, 'count': 7},
 {'_id': 12, 'count': 7},
 {'_id': 19, 'count': 5},
 {'_id': 16, 'count': 5},
 {'_id': 11, 'count': 5},
 {'_id': 18, 'count': 2}]
FAIL: None does not exist in the allowed list of values.


In [29]:
# bedroom large count
pipeline = [
    {"$group": {"_id": "$bedroom_large_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16651752},
 {'_id': 4, 'count': 48463},
 {'_id': 3, 'count': 35081},
 {'_id': 5, 'count': 29178},
 {'_id': 2, 'count': 8174},
 {'_id': 6, 'count': 7157},
 {'_id': 0, 'count': 6763},
 {'_id': 7, 'count': 1541},
 {'_id': 1, 'count': 1351},
 {'_id': 8, 'count': 749},
 {'_id': 10, 'count': 47},
 {'_id': 9, 'count': 4},
 {'_id': 20, 'count': 3}]
FAIL: None does not exist in the allowed list of values.


In [26]:
# average bedroom size
pipeline = [
    {"$group": {"_id": "$average_bedroom_size", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15440107},
 {'_id': 'Average', 'count': 1217657},
 {'_id': 'Large', 'count': 132499}]
FAIL: None does not exist in the allowed list of values.


In [61]:
# construction year 
pipeline = [
    {"$group": {"_id": "$construction_year", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.property_structures.aggregate(pipeline)))

[{'_id': None, 'count': 14419353},
 {'_id': 1980, 'count': 113416},
 {'_id': 1970, 'count': 111641},
 {'_id': 1960, 'count': 94598},
 {'_id': 1985, 'count': 85849},
 {'_id': 1975, 'count': 80340},
 {'_id': 1950, 'count': 79674},
 {'_id': 1990, 'count': 73878},
 {'_id': 2000, 'count': 60554},
 {'_id': 1995, 'count': 55382},
 {'_id': 1965, 'count': 54101},
 {'_id': 2005, 'count': 38790},
 {'_id': 2002, 'count': 37806},
 {'_id': 2003, 'count': 36714},
 {'_id': 2004, 'count': 36464},
 {'_id': 1998, 'count': 35826},
 {'_id': 1955, 'count': 35173},
 {'_id': 2006, 'count': 34058},
 {'_id': 2010, 'count': 33772},
 {'_id': 2007, 'count': 33704},
 {'_id': 2001, 'count': 33200},
 {'_id': 1999, 'count': 32886},
 {'_id': 1994, 'count': 32332},
 {'_id': 2008, 'count': 32259},
 {'_id': 1992, 'count': 31272},
 {'_id': 1996, 'count': 30978},
 {'_id': 2009, 'count': 30034},
 {'_id': 1997, 'count': 29963},
 {'_id': 1993, 'count': 29473},
 {'_id': 1988, 'count': 27203},
 {'_id': 1989, 'count': 27121},
 {'

In [158]:
# distance to bushland category 
pipeline = [
    {"$group": {"_id": "$distance_to_bushland_category", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, "Within 15m", "Between 16m and 100m", "More than 100m"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16705386},
 {'_id': 'Within 15m', 'count': 11008},
 {'_id': 'Between 16m and 100m', 'count': 118},
 {'_id': 'More than 100m', 'count': 68}]
SUCCESS


In [134]:
# has solar panels
pipeline = [
    {"$group": {"_id": "$has_solar_panels", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16714280},
 {'_id': False, 'count': 2289},
 {'_id': True, 'count': 11}]
SUCCESS


In [64]:
# floor area 
pipeline = [
    {"$group": {"_id": "$floor_area", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.property_structures.aggregate(pipeline)))

[{'_id': None, 'count': 16716580}]


In [32]:
# verandah count 
pipeline = [
    {"$group": {"_id": "$verandah_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15392166},
 {'_id': 0, 'count': 749851},
 {'_id': 1, 'count': 522484},
 {'_id': 2, 'count': 101919},
 {'_id': 3, 'count': 14541},
 {'_id': 4, 'count': 8674},
 {'_id': 5, 'count': 356},
 {'_id': 6, 'count': 272}]
FAIL: None does not exist in the allowed list of values.


In [39]:
# lounge room count 
pipeline = [
    {"$group": {"_id": "$lounge_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 103): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16677457},
 {'_id': 1, 'count': 98505},
 {'_id': 0, 'count': 7177},
 {'_id': 2, 'count': 6893},
 {'_id': 3, 'count': 225},
 {'_id': 4, 'count': 6}]
FAIL: None does not exist in the allowed list of values.


In [38]:
# family room count 
pipeline = [
    {"$group": {"_id": "$family_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 102): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16652055},
 {'_id': 0, 'count': 88121},
 {'_id': 1, 'count': 47716},
 {'_id': 2, 'count': 1736},
 {'_id': 3, 'count': 621},
 {'_id': 4, 'count': 10},
 {'_id': 6, 'count': 2},
 {'_id': 5, 'count': 2}]
FAIL: None does not exist in the allowed list of values.


In [40]:
# game room count 
pipeline = [
    {"$group": {"_id": "$game_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 104): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16677457},
 {'_id': 0, 'count': 93598},
 {'_id': 1, 'count': 18780},
 {'_id': 2, 'count': 411},
 {'_id': 3, 'count': 15},
 {'_id': 4, 'count': 2}]
FAIL: None does not exist in the allowed list of values.


In [41]:
# sun room count 
pipeline = [
    {"$group": {"_id": "$sun_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 105): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16677457},
 {'_id': 0, 'count': 103356},
 {'_id': 1, 'count': 9245},
 {'_id': 2, 'count': 191},
 {'_id': 3, 'count': 14}]
FAIL: None does not exist in the allowed list of values.


In [33]:
# dining room count 
pipeline = [
    {"$group": {"_id": "$dining_room_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16671440},
 {'_id': 1, 'count': 90254},
 {'_id': 0, 'count': 26224},
 {'_id': 2, 'count': 2316},
 {'_id': 3, 'count': 26},
 {'_id': 7, 'count': 1},
 {'_id': 6, 'count': 1},
 {'_id': 4, 'count': 1}]
FAIL: None does not exist in the allowed list of values.


In [31]:
# balcony count 
pipeline = [
    {"$group": {"_id": "$balcony_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15392170},
 {'_id': 0, 'count': 914389},
 {'_id': 1, 'count': 381184},
 {'_id': 2, 'count': 83176},
 {'_id': 3, 'count': 14281},
 {'_id': 4, 'count': 3718},
 {'_id': 5, 'count': 709},
 {'_id': 6, 'count': 636}]
FAIL: None does not exist in the allowed list of values.


In [30]:
# bathroom count 
pipeline = [
    {"$group": {"_id": "$bathroom_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.property_structures.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15392121},
 {'_id': 2, 'count': 666573},
 {'_id': 1, 'count': 489720},
 {'_id': 3, 'count': 182762},
 {'_id': 4, 'count': 40333},
 {'_id': 0, 'count': 18736},
 {'_id': 20, 'count': 9},
 {'_id': 5, 'count': 5},
 {'_id': 6, 'count': 3},
 {'_id': 9, 'count': 1}]
FAIL: None does not exist in the allowed list of values.


Properties collection

In [9]:
collection = db['properties']

In [15]:
# has inground pool 
pipeline = [
    {"$group": {"_id": "$has_inground_pool", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.properties.aggregate(pipeline)))

[{'_id': None, 'count': 15414145},
 {'_id': False, 'count': 1168274},
 {'_id': True, 'count': 207844}]


In [16]:
# garage count
pipeline = [
    {"$group": {"_id": "$garage_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15368582},
 {'_id': 2, 'count': 542714},
 {'_id': 0, 'count': 437734},
 {'_id': 1, 'count': 377719},
 {'_id': 3, 'count': 42557},
 {'_id': 4, 'count': 15654},
 {'_id': 6, 'count': 3216},
 {'_id': 5, 'count': 2084},
 {'_id': 7, 'count': 2},
 {'_id': 10, 'count': 1}]
FAIL: None does not exist in the allowed list of values.


In [155]:
# garage types
pipeline = [
    {"$group": {"_id": "$garage_types", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'Brick Garage', 'Fibro Garage', 'Metal Garage']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16708335},
 {'_id': ['Brick Garage'], 'count': 4694},
 {'_id': ['Metal Garage'], 'count': 2494},
 {'_id': ['Fibro Garage'], 'count': 927},
 {'_id': ['Metal Garage', 'Brick Garage'], 'count': 79},
 {'_id': ['Metal Garage', 'Fibro Garage'], 'count': 38},
 {'_id': ['Brick Garage', 'Fibro Garage'], 'count': 12},
 {'_id': ['Metal Garage', 'Brick Garage', 'Fibro Garage'], 'count': 1}]
FAIL: ['Brick Garage'] does not exist in the allowed list of values.
FAIL: ['Metal Garage'] does not exist in the allowed list of values.
FAIL: ['Fibro Garage'] does not exist in the allowed list of values.
FAIL: ['Metal Garage', 'Brick Garage'] does not exist in the allowed list of values.
FAIL: ['Metal Garage', 'Fibro Garage'] does not exist in the allowed list of values.
FAIL: ['Brick Garage', 'Fibro Garage'] does not exist in the allowed list of values.
FAIL: ['Metal Garage', 'Brick Garage', 'Fibro Garage'] does not exist in the allowed list of values.


In [23]:
# carport count 
pipeline = [
    {"$group": {"_id": "$carport_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15412356},
 {'_id': 0, 'count': 964596},
 {'_id': 1, 'count': 270638},
 {'_id': 2, 'count': 122817},
 {'_id': 3, 'count': 12609},
 {'_id': 4, 'count': 5444},
 {'_id': 6, 'count': 1019},
 {'_id': 5, 'count': 784}]
FAIL: None does not exist in the allowed list of values.


In [22]:
# workshop count 
pipeline = [
    {"$group": {"_id": "$workshop_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16677457},
 {'_id': 0, 'count': 80508},
 {'_id': 1, 'count': 29019},
 {'_id': 2, 'count': 2771},
 {'_id': 3, 'count': 376},
 {'_id': 4, 'count': 132}]
FAIL: None does not exist in the allowed list of values.


In [144]:
# has tennis courts 
pipeline = [
    {"$group": {"_id": "$has_tennis_court", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15348343},
 {'_id': False, 'count': 1361727},
 {'_id': True, 'count': 6510}]
SUCCESS


In [145]:
# has pool 
pipeline = [
    {"$group": {"_id": "$has_pool", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, False, True]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16324343},
 {'_id': False, 'count': 362727},
 {'_id': True, 'count': 29510}]
SUCCESS


In [78]:
# has spa 
pipeline = [
    {"$group": {"_id": "$has_spa", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.properties.aggregate(pipeline)))

[{'_id': None, 'count': 16394503},
 {'_id': False, 'count': 314594},
 {'_id': True, 'count': 7483}]


In [146]:
# slope 
pipeline = [
    {"$group": {"_id": "$slope", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, "Flat/Gentle", "Severe"]

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15366570},
 {'_id': 'Flat/Gentle', 'count': 1263030},
 {'_id': 'Severe', 'count': 86901},
 {'_id': 'Moderate', 'count': 79}]
FAIL: Moderate does not exist in the allowed list of values.


In [20]:
# storage small count 
pipeline = [
    {"$group": {"_id": "$storage_small_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15368577},
 {'_id': 0, 'count': 918360},
 {'_id': 1, 'count': 444856},
 {'_id': 2, 'count': 51505},
 {'_id': 3, 'count': 5844},
 {'_id': 4, 'count': 826},
 {'_id': 5, 'count': 180},
 {'_id': 6, 'count': 115}]
FAIL: None does not exist in the allowed list of values.


In [21]:
# storage large count 
pipeline = [
    {"$group": {"_id": "$storage_large_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 15368577},
 {'_id': 0, 'count': 1200028},
 {'_id': 1, 'count': 199661},
 {'_id': 2, 'count': 17428},
 {'_id': 3, 'count': 2900},
 {'_id': 4, 'count': 939},
 {'_id': 6, 'count': 481},
 {'_id': 5, 'count': 249}]
FAIL: None does not exist in the allowed list of values.


In [151]:
# flat_type
pipeline = [
    {"$group": {"_id": "$flat_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'ANTENNA', 'APARTMENT', 'AUTOMATED TELLER MACHINE', 'BLOCK', 'BOATSHED', 'BUILDING', 
                'BUNGALOW', 'CAGE', 'CARPARK', 'CARSPACE', 'CLUB', 'COOLROOM', 'COTTAGE', 'DUPLEX', 'FACTORY', 'FLAT',
                'GARAGE', 'HALL', 'HOUSE', 'KIOSK', 'LOFT', 'LOT', 'MAISONETTE', 'MARINE BERTH', 'OFFICE', 
                'PENTHOUSE', 'REAR', 'RESERVE', 'ROOM', 'SECTION', 'SHED', 'SHOP', 'SHOWROOM', 'SIGN', 'SITE', 
                'STALL', 'STORE', 'STRATA UNIT', 'STUDIO', 'SUBSTATION', 'SUITE', 'TENANCY', 'TOWER', 'TOWNHOUSE',
                'UNIT', 'VAULT', 'VILLA', 'WARD', 'WAREHOUSE', 'WORKSHOP']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 11651478},
 {'_id': 'UNIT', 'count': 4497109},
 {'_id': 'SHOP', 'count': 181597},
 {'_id': 'FLAT', 'count': 132266},
 {'_id': 'SUITE', 'count': 81876},
 {'_id': 'APARTMENT', 'count': 38463},
 {'_id': 'CARSPACE', 'count': 28136},
 {'_id': 'VILLA', 'count': 17647},
 {'_id': 'SITE', 'count': 16600},
 {'_id': 'TENANCY', 'count': 9204},
 {'_id': 'ROOM', 'count': 8372},
 {'_id': 'FACTORY', 'count': 8319},
 {'_id': 'TOWNHOUSE', 'count': 8309},
 {'_id': 'OFFICE', 'count': 7790},
 {'_id': 'SHED', 'count': 6134},
 {'_id': 'HOUSE', 'count': 4541},
 {'_id': 'KIOSK', 'count': 3370},
 {'_id': 'DUPLEX', 'count': 2806},
 {'_id': 'STRATA UNIT', 'count': 2418},
 {'_id': 'STORE', 'count': 1751},
 {'_id': 'COTTAGE', 'count': 1549},
 {'_id': 'WAREHOUSE', 'count': 1275},
 {'_id': 'MARINE BERTH', 'count': 1043},
 {'_id': 'CARPARK', 'count': 866},
 {'_id': 'BOATSHED', 'count': 693},
 {'_id': 'REAR', 'count': 416},
 {'_id': 'LOT', 'count': 406},
 {'_id': 'AUTOMATED TELLER MACHINE', 'cou

In [152]:
# level_type 
pipeline = [
    {"$group": {"_id": "$level_type", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'BASEMENT', 'FLOOR', 'GROUND', 'LEVEL', 'LOBBY', 'LOWER GROUND FLOOR', 'MEZZANINE', 'PARKING',
                'ROOFTOP', 'UPPER GROUND FLOOR']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16461039},
 {'_id': 'LEVEL', 'count': 221426},
 {'_id': 'FLOOR', 'count': 32031},
 {'_id': 'GROUND', 'count': 1203},
 {'_id': 'BASEMENT', 'count': 579},
 {'_id': 'LOWER GROUND FLOOR', 'count': 193},
 {'_id': 'MEZZANINE', 'count': 50},
 {'_id': 'UPPER GROUND FLOOR', 'count': 34},
 {'_id': 'ROOFTOP', 'count': 12},
 {'_id': 'LOBBY', 'count': 10},
 {'_id': 'PARKING', 'count': 3}]
SUCCESS


In [17]:
# level number 
pipeline = [
    {"$group": {"_id": "$level_number", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16538806},
 {'_id': '1', 'count': 52743},
 {'_id': '2', 'count': 33020},
 {'_id': '3', 'count': 22168},
 {'_id': '4', 'count': 16615},
 {'_id': '5', 'count': 13754},
 {'_id': '6', 'count': 11464},
 {'_id': '7', 'count': 9920},
 {'_id': '8', 'count': 8153},
 {'_id': '9', 'count': 7216},
 {'_id': '10', 'count': 6316},
 {'_id': '11', 'count': 5530},
 {'_id': '12', 'count': 4835},
 {'_id': '13', 'count': 4122},
 {'_id': '14', 'count': 3735},
 {'_id': '15', 'count': 3343},
 {'_id': '16', 'count': 3061},
 {'_id': '17', 'count': 2878},
 {'_id': '18', 'count': 2642},
 {'_id': '19', 'count': 2369},
 {'_id': '20', 'count': 2244},
 {'_id': '21', 'count': 2007},
 {'_id': '22', 'count': 1872},
 {'_id': '23', 'count': 1785},
 {'_id': '24', 'count': 1663},
 {'_id': '25', 'count': 1591},
 {'_id': '26', 'count': 1547},
 {'_id': '27', 'count': 1451},
 {'_id': '28', 'count': 1357},
 {'_id': 'B', 'count': 1215},
 {'_id': 'A', 'count': 1190},
 {'_id': '29', 'count': 1180},
 {'_id': 

In [153]:
# street suffix 
pipeline = [
    {"$group": {"_id": "$street_suffix", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []
allowed_list = [None, 'CENTRAL', 'DEVIATION', 'EAST', 'EXTENSION', 'MALL', 'NORTH', 'NORTH WEST', 'ON', 'SOUTH', 
                'SOUTH EAST', 'SOUTH WEST', 'WEST']

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in allowed_list: 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16614793},
 {'_id': 'WEST', 'count': 26375},
 {'_id': 'NORTH', 'count': 25510},
 {'_id': 'EAST', 'count': 24568},
 {'_id': 'SOUTH', 'count': 24338},
 {'_id': 'EXTENSION', 'count': 375},
 {'_id': 'MALL', 'count': 328},
 {'_id': 'CENTRAL', 'count': 263},
 {'_id': 'SOUTH EAST', 'count': 13},
 {'_id': 'DEVIATION', 'count': 11},
 {'_id': 'NORTH WEST', 'count': 4},
 {'_id': 'SOUTH WEST', 'count': 1},
 {'_id': 'ON', 'count': 1}]
SUCCESS


In [18]:
# dwellings count
pipeline = [
    {"$group": {"_id": "$dwellings_count", "count": {"$sum": 1}}}, 
    {"$sort": SON([("count", -1), ("_id", -1)])}
]


test_list = list(db.properties.aggregate(pipeline))
value_list = []

pprint.pprint(test_list)
for x in range(0, len(test_list)): 
    value_list.append(test_list[x].get('_id'))

x = 1
for value in value_list: 
    if value not in range(0, 101): 
        print("FAIL: " + str(value) + " does not exist in the allowed list of values.")
        x = 0

if x==1:
    print("SUCCESS")

[{'_id': None, 'count': 16770465},
 {'_id': 0, 'count': 19796},
 {'_id': 81, 'count': 1},
 {'_id': 12, 'count': 1}]
FAIL: None does not exist in the allowed list of values.
